In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:14:16 of 0:14:17 remaining]
Chain 1:  10% [0:00:57 of 0:01:03 remaining]
Chain 1:  20% [0:01:01 of 0:01:16 remaining]
Chain 1:  30% [0:00:55 of 0:01:18 remaining]
Chain 1:  40% [0:00:48 of 0:01:21 remaining]
Chain 1:  50% [0:00:42 of 0:01:24 remaining]
Chain 1:  60% [0:00:35 of 0:01:27 remaining]
Chain 1:  70% [0:00:27 of 0:01:28 remaining]
Chain 1:  80% [0:00:18 of 0:01:28 remaining]
Chain 1:  90% [0:00:09 of 0:01:28 remaining]
Chain 1: 100% [0:00:00 of 0:01:28 remaining]

Chain 2:   0% [0:00:03 of 0:00:03 remaining]
Chain 2:  10% [0:00:49 of 0:00:54 remaining]
Chain 2:  20% [0:00:52 of 0:01:05 remaining]
Chain 2:  30% [0:00:51 of 0:01:12 remaining]
Chain 2:  40% [0:00:45 of 0:01:14 remaining]
Chain 2:  50% [0:00:38 of 0:01:16 remaining]
Chain 2:  60% [0:00:30 of 0:01:16 remaining]
Chain 2:  70% [0:00:23 of 0:01:17 remaining]
Chain 2:  80% [0:00:16 of 0:01:18 remaining]
Chain 2:  90% [0:00:08 of 0:01:20 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[28.2885 112.147 0.939126; 28.2313 112.144 0.943934; … ; 25.5487 113.837 0.905787; 25.4677 113.837 0.905787]

[25.5991 114.281 0.889505; 25.1 112.989 0.926058; … ; 29.0954 114.777 0.89114; 24.1398 115.141 0.869977]

[26.0611 113.961 0.902615; 26.4364 111.332 0.961805; … ; 28.0174 114.563 0.878424; 27.3669 114.572 0.881622]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD        Naive SE       MCSE        ESS   
     s2  26.1698159 1.98891981 0.01210418050 0.0195158916 9000.0000
beta[1] 113.8483342 1.92055835 0.01168814592 0.0355130042 2924.6923
beta[2]   0.9056834 0.04220359 0.00025684286 0.0007767686 2951.9939

Quantiles:
            2.5%        25.0%        50.0%        75.0%        97.5%   
     s2  22.5579926  24.79325256  26.08026024  27.42794139  30.39652842
beta[1] 109.9344211 112.55394472 113.87843406 115.16375247 117.51249546
beta[2]   0.8245353   0.87690603   0.90517924   0.93418898   0.99163024



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*